In [4]:
# Import libraries and modules
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
sys.path.append(os.path.join(Path.cwd(),'../'))
from libs.db import default_engine as engine

In [6]:
# TOTAL PROFIT AND TOTAL SALES BASED ON PRODUCT PROPERTIES
view_query = """
CREATE VIEW products_total_metrics AS (
	SELECT PS.BRAND, ps.product_line, ps.product_class, ps.product_size, 
		ROUND(SUM(TS.LIST_PRICE - TS.STANDARD_COST), 2) AS total_profit,
		COUNT(ts.transaction_id) AS total_sales
	FROM TRANSACTIONS AS TS
	JOIN PRODUCTS AS PS ON PS.PRODUCT_ID = TS.PRODUCT_ID
	GROUP BY PS.BRAND, ps.product_line, ps.product_class, ps.product_size
)
"""
query = """
SELECT * FROM products_total_metrics ptm
ORDER BY ptm.total_profit DESC
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,brand,product_line,product_class,product_size,total_profit,total_sales
0,WeareA2B,Standard,medium,medium,1826064.01,1949
1,Solex,Standard,medium,medium,1092269.90,2022
2,Trek Bicycles,Standard,medium,large,973246.87,673
3,OHM Cycles,Standard,medium,medium,690762.56,1079
4,WeareA2B,Touring,medium,large,636982.50,430


In [10]:
# SALES PER CUSTOMER AGE
query = """
SELECT
	DATE_PART('year', transaction_date) - DATE_PART('year', birth_date) AS customer_age,
	COUNT(ts.transaction_id) as total_sales,
	ROUND(SUM(ts.list_price), 2) as total_money_spend
FROM TRANSACTIONS AS TS
INNER JOIN CURRENT_CUSTOMERS AS CC ON TS.CUSTOMER_ID = CC.CUSTOMER_ID
GROUP BY customer_age
HAVING 
	DATE_PART('year', transaction_date) - DATE_PART('year', birth_date) BETWEEN 1 AND 90
ORDER BY total_sales DESC
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,customer_age,total_sales,total_money_spend
0,39.0,1106,1210806.30
1,40.0,1053,1128577.21
2,41.0,766,850013.66
3,43.0,748,798060.82
4,37.0,627,704765.45


In [11]:
# BEST SELLING PRODUCT BY STATE
query = """
SELECT
	CC.STATE,
	p.brand,
	p.product_line,
	COUNT(TS.TRANSACTION_ID) AS total_sales
FROM CURRENT_CUSTOMERS AS CC
JOIN TRANSACTIONS AS TS ON TS.CUSTOMER_ID = CC.CUSTOMER_ID
JOIN products AS p ON ts.product_id = p.product_id
GROUP BY
	CC.STATE,
	p.brand,
	p.product_line
HAVING cc.state NOTNULL 
ORDER BY cc.state, total_sales DESC
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,state,brand,product_line,total_sales
0,New South Wales,Solex,Standard,99
1,New South Wales,Giant Bicycles,Standard,61
2,New South Wales,OHM Cycles,Standard,59
3,New South Wales,WeareA2B,Standard,57
4,New South Wales,Norco Bicycles,Standard,41


In [12]:
# TOTAL SALES AND PROFIT BY STATE
query = """
SELECT
	cc.state,
	COUNT(TS.PRODUCT_ID) AS total_sales,
	ROUND(SUM(TS.LIST_PRICE - TS.STANDARD_COST), 2) AS total_profit
FROM TRANSACTIONS AS TS
JOIN CURRENT_CUSTOMERS AS CC ON CC.CUSTOMER_ID = TS.CUSTOMER_ID
WHERE cc.state NOTNULL
GROUP BY CC.state
ORDER BY total_sales DESC
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,state,total_sales,total_profit
0,NSW,10083,5537477.04
1,VIC,4509,2527017.50
2,QLD,4222,2341816.45
3,New South Wales,480,252863.81
4,Victoria,479,254582.60


In [13]:
# WEALTH SEGMENT VS PRODUCT LINE
# No se aprecia correlacion
query = """
SELECT
	CC.WEALTH_SEGMENT,
	PS.brand,
	COUNT(TS.TRANSACTION_ID) as total_sales
FROM CURRENT_CUSTOMERS AS CC
JOIN TRANSACTIONS AS TS ON TS.CUSTOMER_ID = CC.CUSTOMER_ID
JOIN PRODUCTS AS PS ON PS.PRODUCT_ID = TS.PRODUCT_ID
GROUP BY cc.WEALTH_SEGMENT, PS.brand
ORDER BY
	CC.WEALTH_SEGMENT,
	total_sales DESC
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,wealth_segment,brand,total_sales
0,Affluent Customer,Solex,1040
1,Affluent Customer,Giant Bicycles,827
2,Affluent Customer,WeareA2B,777
3,Affluent Customer,Trek Bicycles,764
4,Affluent Customer,OHM Cycles,750


In [14]:
# TOP 10 CUSTOMERS BRAND AND PRODUCT LINE PREFERENCE
query = """
SELECT
	CC."name",
	CC.past_3_years_bike_related_purchases,
	mode() within group(ORDER BY PS.brand) as brand_preference,
	mode() within group(ORDER BY PS.product_line) as product_line_preference
FROM CURRENT_CUSTOMERS AS CC
JOIN TRANSACTIONS AS TS ON TS.customer_id = CC.customer_id
JOIN products AS PS ON PS.product_id = ts.product_id
GROUP BY CC.customer_id
ORDER BY past_3_years_bike_related_purchases DESC
LIMIT 10
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,name,past_3_years_bike_related_purchases,brand_preference,product_line_preference
0,Leesa Castleman,99,WeareA2B,Standard
1,Karin Burkill,99,Giant Bicycles,Road
2,Wendall McKeand,99,Solex,Standard
3,Kiley Canaan,99,Giant Bicycles,Standard
4,Hamlin Odams,99,WeareA2B,Standard


In [15]:
# WEALTH SEGMENT AND JOB TITLE PAST 3 YEARS PURCHASES
query = """
SELECT
	cc.wealth_segment,
	CC.job_title,
	avg(CC.past_3_years_bike_related_purchases) AS avg_past_3_years_bike_related_purchases
from current_customers as cc
group by job_title, wealth_segment
order by
	avg_past_3_years_bike_related_purchases desc,
	cc.wealth_segment,
	job_title
limit 20
"""
df_results = pd.read_sql_query(sql=query, con=engine)
df_results.head()

,wealth_segment,job_title,avg_past_3_years_bike_related_purchases
0,High Net Worth,Automation Specialist IV,98.0
1,Mass Customer,Systems Administrator IV,97.0
2,Affluent Customer,Database Administrator III,96.0
3,Affluent Customer,Database Administrator IV,96.0
4,Affluent Customer,Software Test Engineer IV,96.0
